In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 4.8 MB/s eta 0:00:00


In [ ]:
just_checking_integrity=False

In [ ]:
import gc
import pandas as pd
import scipy.sparse as sps
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
URM_test=sps.load_npz("./drive/MyDrive/Recommendation_system/dataset/processed_data/URM_test_leaderboard.npz")

In [ ]:
URM_test.shape

(4439758, 23692)

In [ ]:
np.ravel(URM_test.sum(axis=1)).shape

(4439758,)

In [ ]:
Session_ids=np.argwhere(np.ravel(URM_test.sum(axis=1)))

In [ ]:
is_sorted = lambda a: np.all(a[:-1] <= a[1:])

In [ ]:
is_sorted(Session_ids)

True

In [ ]:
dataset=None
dataset_max=None
#["UCF","ICBF","ICF","Graph","TopPop100","TopPop50","TopPop15"] # "UCF_W","Graph_W","ICF_W","UCF_WW","ICBF_WW","Graph_WW","ICF_WW", "ICBF_W"
for file in ["Graph"]:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"./drive/MyDrive/Recommendation_system/dataset/candidates/traditional_recs/leaderboard/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    if dataset is None:
        dataset=df_feats
        dataset_max=df_max
    else:
        dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
        dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


del df_feats
del df_max

0
loading Graph
8315730
loaded Graph


In [ ]:
ids=np.sort(dataset["Session_Id"].unique())
map_ids=pd.DataFrame()
map_ids["Session_Id"]=ids
if just_checking_integrity:
    map_ids["Real_Id"]=Session_ids[:len(ids)]
else:
    map_ids["Real_Id"]=Session_ids

In [ ]:
dataset_max=dataset_max.merge(map_ids,on="Session_Id",how="left")
del dataset_max["Session_Id"]
dataset_max.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [ ]:
dataset=dataset.merge(map_ids,on="Session_Id",how="left")
del dataset["Session_Id"]
dataset.rename(columns={"Real_Id":"Session_Id"},inplace=True)

In [ ]:
# import ast
# submission_df=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/GRU/leaderboard/candidates.csv")
# submission_df["score"]=submission_df["score"].map(ast.literal_eval)
# submission_df["code"]=submission_df["code"].map(ast.literal_eval)
# submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
# submission_df = submission_df.explode(['code','score', 'rank'])
# submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
# submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [ ]:
# import ast
# submission_df_tr=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/Transformer/leaderboard/candidates.csv")
# submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
# submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
# submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
# submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
# submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
# submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [ ]:
import ast
submission_df_LSTM=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/LSTM/leaderboard/candidates.csv")
submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [ ]:
# dataset=dataset.merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
# del submission_df
# dataset=dataset.merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
# del submission_df_tr
dataset=dataset.merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")


In [ ]:
dataset=dataset.merge(dataset_max,on=["Session_Id"],how="left")
del dataset_max

In [ ]:
dataset.fillna(0,inplace=True)

In [ ]:
dataset_macro_features = pd.read_csv('./drive/MyDrive/Recommendation_system/dataset/processed_data/macro_feats_NN_leaderboard.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items'])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)
dataset = dataset.merge(dataset_macro_features, on='Session_Id', how='left')

In [ ]:
# dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

In [ ]:
dataset.head()

,Item_ID,Score_Graph,Session_Id,score_LSTM,rank_LSTM,Max_Score_Graph,length_of_session_seconds,avg_time_spent_per_item_seconds,variance_time_spent_per_item_seconds,n_seen_items,n_unique_items,session_similarity,session_similarity_uniques,session_similarity_feats,session_similarity_feats_uniques
0,3,0.010701,26,0.000000,0.0,2.522003,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,21,0.019740,26,0.000000,0.0,2.522003,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,52,0.087397,26,0.002269,33.0,2.522003,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,80,0.045723,26,0.001017,76.0,2.522003,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,159,0.020845,26,0.000000,0.0,2.522003,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


In [ ]:
gc.collect()

2365

In [ ]:
dataset=dataset.sort_values("Session_Id")

In [ ]:
gc.collect()

0

In [ ]:
k=10

In [ ]:
result = []

In [ ]:
from catboost import CatBoost

result = []

for index in range(k):
    model = CatBoost().load_model(f'./drive/MyDrive/Recommendation_system/rankers/catboost/graph_and_lstm/model_{index}.cbm')
    result.append(model.predict(dataset.drop(["Session_Id", "Item_ID"], axis=1)))

# combine scores
result = np.array(result)
result


array([[-0.65235259, -0.54526015, -0.58574847, ...,  1.24170337,
        -0.60189106, -1.00164954],
       [-0.64573021, -0.54348334, -0.62256137, ...,  1.22081764,
        -0.58202189, -0.94297621],
       [-0.6406867 , -0.55991001, -0.59979666, ...,  1.26059163,
        -0.59795848, -0.99672964],
       ...,
       [-0.61065273, -0.54139553, -0.57705644, ...,  1.24187366,
        -0.61687852, -1.02416712],
       [-0.68330679, -0.58464443, -0.6285023 , ...,  1.29895459,
        -0.63405767, -1.0869768 ],
       [-0.74231149, -0.59905294, -0.70205196, ...,  1.239855  ,
        -0.60286473, -1.02449124]])

In [ ]:
result.shape

(10, 10656923)

In [ ]:
dataset["score"]=np.mean(result ,axis=0)
# dataset["score"]=np.median(result ,axis=0)

#dataset["score"]=dataset[[f"score_{i}" for i in range(k)]].apply(lambda x:np.median(x),axis=1)

In [ ]:
dataset=dataset[["Session_Id","Item_ID","score"]]
gc.collect()

0

In [ ]:
dataset.head()

,Session_Id,Item_ID,score
0,26,3,-0.677059
126,26,3944,-0.574591
127,26,4015,-0.629624
128,26,4058,-0.074511
129,26,4065,-0.677059


In [ ]:
dataset=dataset.groupby(['Session_Id']).agg(list).reset_index()

In [ ]:
def get_indices(x):
    x=np.array(x)
    indices_unsorted=np.argpartition(x, -100)[-100:]
    x=x[indices_unsorted]
    indices=indices_unsorted[np.argsort(-x)]
    return indices

In [ ]:
dataset["indices"]=dataset["score"].apply(lambda x: get_indices(x))

In [ ]:
dataset.iloc[0]

Session_Id                                                   26
Item_ID       [3, 3944, 4015, 4058, 4065, 4166, 4184, 4212, ...
score         [-0.6770594798968378, -0.574590884661099, -0.6...
indices       [143, 34, 104, 19, 79, 48, 37, 49, 22, 114, 11...
Name: 0, dtype: object

In [ ]:
dataset.iloc[0]["Item_ID"][126]

159

In [ ]:
def get_items(x):
    items=np.array(x[0])
    return items[x[1]]

In [ ]:
dataset["predictions"]=dataset[["Item_ID","indices"]].apply(get_items,axis=1)

In [ ]:
dataset=dataset[["Session_Id","predictions"]]
dataset["rank"]=dataset.apply(lambda x:tuple([rank for rank in range(1, 101)]) ,axis=1)

In [ ]:
dataset

,Session_Id,predictions,rank
0,26,"[606, 3646, 1008, 4685, 4952, 4841, 3720, 4881...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"[772, 1827, 270, 3400, 3229, 4878, 3656, 2734,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"[3398, 2289, 4986, 1181, 2235, 1484, 3203, 786...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"[2378, 4900, 4155, 534, 3922, 1633, 2369, 3201...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"[3172, 1004, 3180, 1518, 25, 1768, 3022, 3309,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
...,...,...,...
49995,4439446,"[4810, 3526, 2258, 1468, 949, 2605, 4835, 3084...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49996,4439458,"[2877, 3880, 3155, 867, 3550, 3026, 1183, 2919...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49997,4439550,"[4779, 4846, 625, 1901, 2431, 2835, 2638, 2266...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
49998,4439653,"[2596, 1015, 42, 2266, 2638, 3531, 360, 36, 36...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [ ]:
dataset["predictions"]=dataset["predictions"].apply(tuple)

In [ ]:
dataset=dataset[["Session_Id","predictions","rank"]]

In [ ]:
dataset.columns=["session_id","code","rank"]

In [ ]:
dataset.head()

,session_id,code,rank
0,26,"(606, 3646, 1008, 4685, 4952, 4841, 3720, 4881...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200,"(772, 1827, 270, 3400, 3229, 4878, 3656, 2734,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205,"(3398, 2289, 4986, 1181, 2235, 1484, 3203, 786...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495,"(2378, 4900, 4155, 534, 3922, 1633, 2369, 3201...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521,"(3172, 1004, 3180, 1518, 25, 1768, 3022, 3309,...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [ ]:
submission_df = dataset.explode(['code', 'rank']).astype(int)

In [ ]:
mapping= pd.read_csv('./drive/MyDrive/Recommendation_system/dataset/processed_data/map.csv')
submission_df=submission_df.merge(mapping,on="code",how="left")
del submission_df["code"]
submission_df=submission_df[["session_id","item_id","rank"]]

In [ ]:
print(len(submission_df))

5000000


In [ ]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000000 entries, 0 to 4999999
Data columns (total 3 columns):
 #   Column      Dtype
---  ------      -----
 0   session_id  int64
 1   item_id     int64
 2   rank        int64
dtypes: int64(3)
memory usage: 152.6 MB


In [ ]:
submission_df.isna().sum()

session_id    0
item_id       0
rank          0
dtype: int64

In [ ]:
submission_df.to_csv(
    f'./drive/MyDrive/Recommendation_system/rankers/catboost/graph_and_lstm/leaderboard/leaderboard_sub.csv',
    index=False
    )